In [ ]:
# Import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt

# Define the midpoint
midpoint = 3

# Set file and column names
filename = "OppScore.csv"
outcome_name = "outcome"
imp_name = "importance"
sat_name = "satisfaction"
remove_missing = True

In [ ]:
# Define functions

# Opportunity score = Importance + max(Importance − Satisfaction , 0)
def op_score(imp, sat):
    return imp + max(imp - sat, 0)

def count_positives(x, midpoint):
    x = x.dropna()
    return 10 * (sum(x > midpoint) / len(x))

def plot_opp_score(values):
    grey = "#AAAAAA"
    plt.figure(figsize=(8, 8))
    plt.xlim(0, 10)
    plt.ylim(0, 10)
    plt.xlabel("importance")
    plt.ylabel("satisfaction")
    plt.title("Opportunity Landscape")
    plt.axline((0, 0), (10, 10), linestyle='-', color=grey)
    plt.axline((5, 0), (10, 10), linestyle='-', color=grey)
    plt.plot([0, 10], [-11.5, (10-1)/(10-5.5)*10-11.5], linestyle='--', color=grey)
    plt.plot([0, 10], [-14.5, (10-1)/(10-5.5)*10-14.5], linestyle='--', color=grey)
    plt.text(3, 5.8, "Overserved", color=grey)
    plt.text(4, 2, "Appropriately\nServed", color=grey)
    plt.text(6.5, 2, "Underserved", color=grey)
    plt.text(7.5, 4.5, "Opp>10\nSolid Opportunity", color=grey)
    plt.text(7.6, 3, "Opp>12\nHigh Opportunity", color=grey)
    plt.text(7.7, 0.4, "Opp>15\nExtreme Opportunity", color=grey)
    plt.scatter(values['importance'], values['satisfaction'], color='red', marker='o', s=50)
    for i, txt in enumerate(values.index.map(str)):
        plt.annotate(txt, (values['importance'][i+1], values['satisfaction'][i+1]), xytext=(6, -3), textcoords='offset points', color='black', fontweight='bold', fontsize=8)
    plt.show()

In [ ]:
# Read the CSV file
data = pd.read_csv(filename)

# Remove rows with missing values if required
if remove_missing:
    data = data.dropna(subset=[imp_name, sat_name])

imp = data.groupby('outcome')['importance'].apply(lambda x: count_positives(x, midpoint)).reset_index()
sat = data.groupby('outcome')['satisfaction'].apply(lambda x: count_positives(x, midpoint)).reset_index()

values = imp.merge(sat, on="outcome", how="outer")

# Compute opportunity scores
values['oppscore'] = values.apply(lambda row: op_score(row['importance'], row['satisfaction']), axis=1)

# Sort the values by opportunity score
values = values.sort_values(by='oppscore', ascending=False)

# Reset the index
values = values.reset_index(drop=True)
values.index += 1

In [ ]:
# Print the opportunity scores with the outcomes column left aligned and the rest right aligned

# Custom formatter for the outcome column to left align the text
max_len = values['outcome'].str.len().max()
outcome_fmt = lambda x: '{:<{}}'.format(x, max_len)

# Hack to fix stupid alignment of the outcome column header
t = values.to_string(formatters={'outcome': outcome_fmt}, float_format='{:>.2f}'.format)
print(t.replace(' '*(max_len-7)+'outcome', 'outcome'+' '*(max_len-7)))

# Plot the opportunity landscape
plot_opp_score(values)